Code, takes list of file paths for the monthly inflow sheets -> returns final sheet with rows as powiat, date - cols for cumulative inflows. Can then pivot to what is needed

In [1]:
import sys
from pathlib import Path

p = Path.cwd().resolve()
repo_root = next((parent for parent in [p] + list(p.parents) if (parent / ".git").exists()), None)
if repo_root is None:
    raise RuntimeError("Repo root not found. Open the repo folder in VS Code.")

sys.path.insert(0, str(repo_root))
print("Repo root:", repo_root)

Repo root: C:\Users\harri\OneDrive - Imperial College London\Year 3 Group Project\Group_Project_Y3


Imports

In [2]:
import pandas as pd
import numpy as np
import re

Clean Individual df

In [3]:
def clean_piece(x):
    # Cleans multi-level column name piece
    if pd.isna(x):
        return ""
    x = str(x).strip()
    if x.startswith("Unnamed:"):
        return ""
    x = re.sub(r"\s+", "_", x)
    return x

def flatten_column_names(df):
    # Convert multi-level column name to clean format
    new_cols = []
    for a, b, c in df.columns:
        parts = [clean_piece(a), clean_piece(b), clean_piece(c)]
        parts = [p for p in parts if p]  # drop blanks
        col = "_".join(parts)
        new_cols.append(col)
    df = df.copy()
    df.columns = new_cols
    return df

def translate_col(x):
    # Translate single column name according to mapping
    map = {
        "TERYT": "teryt",
        "POWIAT": "powiat",
        "WOJEWÓDZTWO": "voivodeship",
        "WOJEWODZTWO": "voivodeship",
        "ŁĄCZNIE": "total",
        "LACZNIE": "total",
        "KOBIET": "female",       # appears as "KOBIET" in your header
        "KOBIETY": "female",
        "MĘŻCZYZN": "male",       # appears as "MĘŻCZYZN" in your header
        "MEZCZYZN": "male",
        "MĘŻCZYŹNI": "male",
        "MEZCZYZNI": "male",
        "WSZYSTKICH": "all",
        # "ROK_URODZENIA": "birth_year",
        # "ROK": "year",
        # "URODZENIA": "birth",
        "ROK_URODZENIA": "", # just to replace this with none
        "ROK": "",
        "URODZENIA": "",
    } # thank you chat

    parts = x.split("_")
    parts = [map.get(p, p.lower()) for p in parts]

    out = "_".join([p for p in parts if p])
    return out

def translate_columns(df):
    # Translate column names to english
    out = df.copy()
    out.columns = [translate_col(c).replace("...", "1900") for c in out.columns] # the replace just turns ...-1958 to 1900-1958
    return out

# ========================================================

def fill_lt2_equal_share(df, total_col, prefix, lt2_token="<2"):
    # Takes total column and prefix for individual columns which should sum to it
    # Assigns <2 with numeric value equal to share of missing total for that row
    out = df.copy()

    # get individual cols
    cols = [c for c in out.columns if c.startswith(prefix)]
    
    # total as numeric
    out[total_col] = pd.to_numeric(out[total_col], errors="coerce")

    # mask for cells that are "<2"
    lt2_mask = out[cols].astype(str).eq(lt2_token)

    # convert cols to numeric, "<2" becomes NaN
    # cols_num = out[cols].replace(lt2_token, np.nan)
    cols_num = out[cols].mask(lt2_mask)
    cols_num = cols_num.apply(pd.to_numeric, errors="coerce")

    # row sums
    known_sum = cols_num.sum(axis=1, skipna=True)

    # calculate missing share for row
    missing_total = out[total_col] - known_sum
    k = lt2_mask.sum(axis=1)
    share = missing_total.div(k).where(k>0)

    # fill "<2" with computed share
    filled = cols_num.copy()
    for col in cols:
        filled[col] = np.where(lt2_mask[col], share, filled[col]) # if lt2 mask then share, else take existing value
    
    out[cols] = filled
    return out

# ========================================================

def birth_to_age_label(col, ref_year):
    # Convert columns like:
    #     male_2026 -> male_0
    #     male_1900-1958 -> male_68-126   
    # using age = ref_year - birth_year (assuming birth at year start)
    
    m = re.match(r"^(male|female)_(\d{4})(?:-(\d{4}))?$", col)
    if not m:
        return col

    sex, y1, y2 = m.group(1), int(m.group(2)), m.group(3)

    # ie only a single year column
    if y2 is None:
        age = ref_year - y1
        return f"{sex}_{age}"

    # otherwise it is a range column
    y2 = int(y2)
    age_high = ref_year - y1 # y1 lower year -> higher age
    age_low  = ref_year - y2
    return f"{sex}_{age_low}-{age_high}"

def convert_birthyear_cols_to_age(df, date_col="date"):
    # Rename male_* and female_* birth-year columns to age columns using date column year
    out = df.copy()
    ref_year = int(pd.to_datetime(out[date_col]).dt.year.dropna().iloc[0])

    rename_map = {}
    for c in out.columns:
        rename_map[c] = birth_to_age_label(c, ref_year)

    out = out.rename(columns=rename_map)
    return out

# ========================================================

def clean_single_sheet(path, incl_2026=True):
    datestring = path.split("_")[-1].replace(".xlsx", "")
    print(f"Processing {datestring}")

    if datestring[0:4]=="2026" and not incl_2026:
        return None

    # Clean df formatting
    df = pd.read_excel(path, header=[0, 1, 2])
    df = flatten_column_names(df) # flatten multi level column name
    df = translate_columns(df) # translate cols to english
    df["date"] = pd.to_datetime(datestring, format="%Y%m%d") # add column for date

    # Process the <2 and convert to ints
    df = fill_lt2_equal_share(df, "total_female", "female")
    df = fill_lt2_equal_share(df, "total_male", "male")

    # Convert cols to age, i.e. if 2026 male_2026 becomes male_0, this allows stacking of dfs (hopefully) - it doesn't exactly but useful still
    df = convert_birthyear_cols_to_age(df, date_col="date")

    # Return df
    return df

In [11]:
# clean_single_sheet(f"{str(repo_root)}/raw/01_inflow_data/STATYSTYKI_POWIAT_UKR_20220414.xlsx") # Uncomment to see example process

Get all xlsx in folder and process into one stacked df

In [ ]:
folder = repo_root / "raw/03_00_inflow_data"

xlsx_files = [str(p) for p in folder.glob("*.xlsx")]

print(f"Found {len(xlsx_files)} files.")

dfs = []
for path in xlsx_files:
    df = clean_single_sheet(path, incl_2026=True)
    if df is not None:
        dfs.append(df)
    else:
        print("Skipped - 2026 file!")

print(f"\nProcessed {len(dfs)} files.")

df_all = pd.concat(dfs, ignore_index=True, sort=False)

Found 58 files.
Processing 20220414
Processing 20220426
Processing 20220517
Processing 20220601
Processing 20220615
Processing 20220701
Processing 20220714
Processing 20220801
Processing 20220901
Processing 20220915
Processing 20220928
Processing 20221031
Processing 20221130
Processing 20221227
Processing 20230131
Processing 20230228
Processing 20230328
Processing 20230425
Processing 20230530
Processing 20230627
Processing 20230704
Processing 20230711
Processing 20230718
Processing 20230725
Processing 20230801
Processing 20230808
Processing 20230816
Processing 20230822
Processing 20230927
Processing 20231010
Processing 20231114
Processing 20231212
Processing 20240109
Processing 20240213
Processing 20240312
Processing 20240409
Processing 20240514
Processing 20240611
Processing 20240709
Processing 20240813
Processing 20240910
Processing 20241008
Processing 20241112
Processing 20241210
Processing 20250114
Processing 20250211
Processing 20250311
Processing 20250407
Processing 20250513
Proc

Now bin by ages (assuming uniform distribution on overlaps)

In [14]:
def parse_age_col(col):
    # Parse 'male_7' -> ('male', 7, 7)
    #       'female_20-24' -> ('female', 20, 24)
    #       'male_65+' -> ('male', 65, 200)
    # Returns None if not an age column.
    
    m = re.match(r'^(male|female)_(\d+)(?:-(\d+))?$', col)
    if m:
        sex = m.group(1)
        lo = int(m.group(2))
        hi = int(m.group(3)) if m.group(3) else lo
        return sex, lo, hi

    m = re.match(r'^(male|female)_(\d+)\+$', col)
    if m:
        sex = m.group(1)
        lo = int(m.group(2))
        return sex, lo, 200

    return None

def overlap_len(a_lo, a_hi, b_lo, b_hi):
    lo = max(a_lo, b_lo)
    hi = min(a_hi, b_hi)
    return max(0, hi - lo + 1)

def target_colname(sex: str, lo: int, hi: int):
    if lo == hi:
        return f"{sex}_{lo}"
    if hi >= 200:
        return f"{sex}_{lo}+"
    return f"{sex}_{lo}-{hi}"

# ======================================================

def harmonise_age_bins_by_overlap(
    df_all,
    target_bins,
    id_cols=("date", "teryt", "powiat", "voivodeship", "total_all", "total_male", "total_female"),
    sexes=("male", "female"),
    cap_hi=200,
):
    # For each sex and each target bin, sum contributions from all overlapping source age columns
    # in df_all, proportional to overlap width / source width.
    
    out = df_all.copy()

    # collect all source age columns for each sex
    src_by_sex = {sex: [] for sex in sexes}
    for c in out.columns:
        parsed = parse_age_col(c)
        if parsed and parsed[0] in sexes:
            sex, lo, hi = parsed
            hi = min(hi, cap_hi)
            src_by_sex[sex].append((c, lo, hi))

    # prepare result df with just ids
    keep_ids = [c for c in id_cols if c in out.columns]
    res = out.loc[:, keep_ids].copy()

    # build target columns by overlap-summing
    for sex in sexes:
        # numeric view of source cols (missing -> 0)
        src_cols = [c for c, _, _ in src_by_sex[sex]]
        if not src_cols:
            # still create empty target cols - probably shouldnt be any though
            for lo, hi in target_bins:
                res[target_colname(sex, lo, hi)] = np.nan
            continue

        src_vals = out[src_cols].apply(pd.to_numeric, errors="coerce").fillna(0.0)

        for t_lo, t_hi in target_bins:
            t_hi = min(t_hi, cap_hi)
            tcol = target_colname(sex, t_lo, t_hi)

            acc = np.zeros(len(out), dtype=float)

            for c, s_lo, s_hi in src_by_sex[sex]:
                ol = overlap_len(s_lo, s_hi, t_lo, t_hi)
                if ol == 0:
                    continue
                width = (s_hi - s_lo + 1)
                frac = ol / width
                acc += src_vals[c].to_numpy() * frac

            res[tcol] = acc

    return res

In [15]:
TARGET_BINS = (
    [(i, i) for i in range(0, 20)] +
    [(20, 24), (25, 29), (30, 34), (35, 39),
     (40, 44), (45, 49), (50, 54), (55, 59),
     (60, 64), (65, 200)] # set last to 200 for the 65+ col
)

df_harmonised = harmonise_age_bins_by_overlap(df_all, TARGET_BINS)

# set original columns to float for consistency
num_cols = df_harmonised.select_dtypes(include=["number"]).columns
df_harmonised[num_cols] = df_harmonised[num_cols].astype(float)

In [18]:
# Check sum over bins correct
male_targets = [c for c in df_harmonised.columns if c.startswith("male_")]
female_targets = [c for c in df_harmonised.columns if c.startswith("female_")]

df_harmonised["sum_male"] = df_harmonised[male_targets].sum(axis=1) 
df_harmonised["sum_female"] = df_harmonised[female_targets].sum(axis=1)

mask = (df_harmonised["total_female"].round(2) == df_harmonised["sum_female"].round(2))
mask.value_counts()
# df_harmonised[mask==False][["total_female", "sum_female"]]

True    22040
Name: count, dtype: int64

In [17]:
# drop the sum columns and then export to csv
df_harmonised.drop(["sum_male", "sum_female"], axis=1, inplace=True)

df_harmonised.to_csv(f"{str(repo_root)}/cleaned/01_inflow_data/inflow_semi_cleaned.csv", index=False, encoding="utf-8-sig")